In [2]:
!pip install opensmile gdown transformers soundfile --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 996.0/996.0 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.4/138.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.8/324.8 kB 12.3 MB/s eta 0:00:00


In [3]:
import os
import tarfile
from pathlib import Path
import os
import tempfile
import concurrent.futures
from pathlib import Path
import matplotlib.pyplot as plt
import librosa
import librosa.display
from IPython.display import Audio, display
import pandas as pd
import numpy as np
import soundfile as sf
from transformers import pipeline
import torch

# install dataset using gdown
!gdown 1p4ZQOwbHkD2RAvq2K5ekY5gcMi24XLnS

tarball_filename = "ADReSSo21-diagnosis-train.tar"

# extract contents from tarball
with tarfile.open(tarball_filename, 'r:*') as tar:
    tar.extractall(path="./")

# remove tarball after extraction
os.remove(tarball_filename)

Downloading...
From (original): https://drive.google.com/uc?id=1p4ZQOwbHkD2RAvq2K5ekY5gcMi24XLnS
From (redirected): https://drive.google.com/uc?id=1p4ZQOwbHkD2RAvq2K5ekY5gcMi24XLnS&confirm=t&uuid=7a2addbe-598d-4d17-ba6a-8470fded940d
To: /content/ADReSSo21-diagnosis-train.tar
100% 1.75G/1.75G [00:17<00:00, 99.8MB/s]


### Getting train/test Split

In [6]:
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

def get_dataset_paths(root="ADReSSo21/diagnosis/train"):
    root = Path(root)
    return {
        "ad": root / "audio" / "ad",
        "cn": root / "audio" / "cn",
    }

paths = get_dataset_paths()

# file list
records = []
for p, lbl in [(paths["ad"], 1), (paths["cn"], 0)]:
    for wav in sorted(p.glob("*.wav")):
        records.append({"file_id": wav.stem, "label": lbl})

df = pd.DataFrame(records)
print("total files:", len(df))

# split
train_ids, test_ids = train_test_split(
    df["file_id"],
    test_size=0.20,
    stratify=df["label"],
    random_state=23,
)

df["split"] = np.where(df["file_id"].isin(train_ids), "train", "test")
print(df["split"].value_counts())

# save
df.to_csv("split_master.csv", index=False)
print("saved split_master.csv")

total files: 166
split
train    132
test      34
Name: count, dtype: int64
saved split_master.csv
